In [1]:
import mesa
import numpy as np

from Model_Ilya import Microbiome, Soil, Type_a_1, Type_a_2

### Initiate model with bacterium mass drawn from Gaussian distribution

In [2]:
num_samples = 5
mean_weight = 0.28 # pg
variation_coefficient = 0.1 # in paper it is 10 however, may be 10%

def set_initial_mass(mean_weight, variation_coefficient, num_samples=1):

    values = np.random.normal(mean_weight, variation_coefficient*mean_weight, num_samples)

    negative_indices = np.where(values <= 0)[0]

    while len(negative_indices) > 0:

        new_values = np.random.normal(mean_weight, variation_coefficient*mean_weight, len(negative_indices))
        values[negative_indices] = new_values
        negative_indices = np.where(values <= 0)[0]

    return values


mass_sample = set_initial_mass(mean_weight, variation_coefficient, num_samples)

mass_sample

#Adjust the divide function where after division the mass is first randomly assigned
#Then make it split in half from the mother cell -> need to be able to generate mass -> define function growth where nutrients are converted into mass -> define split
#Keep validating these changes

array([0.28516845, 0.27845692, 0.30760758, 0.27697959, 0.27472269])

### Trying new features of the model

In [3]:
#Making 2 steps with in the model

model = Microbiome(10, 10, True, 50, 50)
for i in range(2):
    model.step()


In [4]:
# Check the numbers of agents

model.datacollector.model_vars

{'Type_a_1': [10, 20], 'Type_a_2': [10, 20]}

In [9]:
# Extract the data about mass of one of the bacteria

my_agents = [agent for agent in model.schedule.agents if isinstance(agent, Type_a_2)]
for agent in my_agents:
    # Access the variables of the current agent
    my_var = agent.mass

    # Print the unique ID and variable of the current agent
    print(f"MyAgent {agent.unique_id}: mass={my_var}")

MyAgent 2512: mass=[0.29545318]
MyAgent 2513: mass=[0.27575021]
MyAgent 2514: mass=[0.30769183]
MyAgent 2515: mass=[0.28309994]
MyAgent 2516: mass=[0.29412281]
MyAgent 2517: mass=[0.34571842]
MyAgent 2518: mass=[0.25850028]
MyAgent 2519: mass=[0.22032649]
MyAgent 2520: mass=[0.31651329]
MyAgent 2521: mass=[0.28700324]
MyAgent 2523: mass=[0.2728938]
MyAgent 2524: mass=[0.28480145]
MyAgent 2526: mass=[0.27681028]
MyAgent 2528: mass=[0.24701868]
MyAgent 2531: mass=[0.25825716]
MyAgent 2533: mass=[0.21984311]
MyAgent 2534: mass=[0.30745565]
MyAgent 2536: mass=[0.24695003]
MyAgent 2539: mass=[0.30007568]
MyAgent 2540: mass=[0.2741424]
MyAgent 2543: mass=[0.29634067]
MyAgent 2545: mass=[0.27282693]
MyAgent 2546: mass=[0.26873911]
MyAgent 2547: mass=[0.30194507]
MyAgent 2548: mass=[0.25232377]
MyAgent 2550: mass=[0.29679414]
MyAgent 2552: mass=[0.28690273]
MyAgent 2555: mass=[0.26909021]
MyAgent 2556: mass=[0.30286242]
MyAgent 2558: mass=[0.21638098]
MyAgent 2560: mass=[0.27358996]
MyAgent 25